In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import xgboost as xgb
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric

class ImpactParityXGBoostClassifier:
    def __init__(self, fairness_regularizer_weight=0.3, impact_threshold=0.9, base_estimator_params=None):
        self.fairness_weight = fairness_regularizer_weight
        self.impact_threshold = impact_threshold
        self.base_params = base_estimator_params or {
            'max_depth': 3,
            'learning_rate': 0.1,
            'n_estimators': 200,
            'objective': 'binary:logistic'
        }
        self.model = None
        self.protected_attributes = None
        
    def _compute_impact_parity_penalty(self, y_pred):
        y_pred_binary = (y_pred > 0.5).astype(int)
        privileged_mask = self.protected_attributes == 1
        unprivileged_mask = self.protected_attributes == 0
        
        priv_positive_rate = np.mean(y_pred_binary[privileged_mask])
        unpriv_positive_rate = np.mean(y_pred_binary[unprivileged_mask])
        
        eps = 1e-10
        impact_ratio = unpriv_positive_rate / (priv_positive_rate + eps)
        
        if impact_ratio < self.impact_threshold:
            penalty = (self.impact_threshold - impact_ratio) ** 2
        else:
            penalty = abs(1.0 - impact_ratio)
            
        return penalty
        
    def _fair_objective(self, y_pred, dtrain):
        y_true = dtrain.get_label()
        grad = y_pred - y_true
        hess = y_pred * (1.0 - y_pred)
        fairness_penalty = self._compute_impact_parity_penalty(y_pred)
        grad += self.fairness_weight * fairness_penalty
        return grad, hess
    
    def fit(self, X, y, protected_attributes):
        self.protected_attributes = protected_attributes
        dtrain = xgb.DMatrix(X, label=y)
        params = self.base_params.copy()
        params['objective'] = None
        self.model = xgb.train(params, dtrain, obj=self._fair_objective)
        
    def predict(self, X):
        if self.model is None:
            raise ValueError("Model needs to be fitted first")
        dtest = xgb.DMatrix(X)
        return (self.model.predict(dtest) > 0.5).astype(int)
    
    def predict_proba(self, X):
        if self.model is None:
            raise ValueError("Model needs to be fitted first")
        dtest = xgb.DMatrix(X)
        return self.model.predict(dtest)

def create_aif_dataset(X, y, protected_attribute_name, protected_values):
    df = pd.DataFrame(X.copy())
    df['label'] = y
    df[protected_attribute_name] = protected_values
    
    return BinaryLabelDataset(
        df=df,
        label_names=['label'],
        protected_attribute_names=[protected_attribute_name],
        favorable_label=1.0,
        unfavorable_label=0.0
    )

def evaluate_detailed_metrics(model, X_test, y_test, protected_test, group_name="sex_Male"):
    predictions = model.predict(X_test)
    
    print("Overall Performance Metrics:")
    print(classification_report(y_test, predictions))
    
    dataset_true = create_aif_dataset(X_test, y_test, group_name, protected_test)
    dataset_pred = create_aif_dataset(X_test, predictions, group_name, protected_test)
    
    privileged_groups = [{group_name: 1}]
    unprivileged_groups = [{group_name: 0}]
    
    dataset_metrics = BinaryLabelDatasetMetric(
        dataset_true,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups
    )
    
    class_metrics = ClassificationMetric(
        dataset_true,
        dataset_pred,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups
    )
    
    print("\nFairness Metrics:")
    print(f"Disparate Impact: {dataset_metrics.disparate_impact():.3f}")
    print(f"Statistical Parity Difference: {dataset_metrics.statistical_parity_difference():.3f}")
    print(f"Equal Opportunity Difference: {class_metrics.equal_opportunity_difference():.3f}")
    print(f"Average Odds Difference: {class_metrics.average_odds_difference():.3f}")

def prepare_data(filepath):
    df = pd.read_csv(filepath)
    
    use_df = df[['sex','age_cat','race',
                'juv_fel_count','juv_misd_count','juv_other_count','priors_count',
                'c_charge_degree', 'decile_score', 'v_decile_score', 'score_text', 
                'v_score_text', 'is_recid', 'is_violent_recid']]

    categorical_columns = ['sex', 'age_cat', 'c_charge_degree']
    encoded_df = pd.get_dummies(use_df, columns=categorical_columns, drop_first=True)
    encoded_df = pd.get_dummies(encoded_df, columns=['race'], drop_first=False)

    score_mapping = {'Low': 0, 'Medium': 1, 'High': 2}
    encoded_df['score_text'] = encoded_df['score_text'].map(score_mapping)
    encoded_df['v_score_text'] = encoded_df['v_score_text'].map(score_mapping)

    X = encoded_df.drop(columns=['decile_score', 'v_decile_score', 'score_text', 
                               'v_score_text', 'is_recid', 'is_violent_recid'])
    y = encoded_df['is_recid']

    # For gender bias
    # protected_attributes = encoded_df['sex_Male'].values

    # For age bias
    protected_attributes = encoded_df['age_cat_Greater than 45'].values

    # For race bias
    # protected_attributes = encoded_df['race_African-American'].values

    return train_test_split(X, y, protected_attributes, test_size=0.3, random_state=42)

def main():
    X_train, X_test, y_train, y_test, protected_train, protected_test = prepare_data('compas-scores-two-years.csv')

    print("\n=== Training Fair XGBoost Model ===")
    fair_xgb = ImpactParityXGBoostClassifier(
        fairness_regularizer_weight=0.3,
        impact_threshold=0.9,
        base_estimator_params={
            'max_depth': 3,
            'learning_rate': 0.1,
            'n_estimators': 200,
            'random_state': 42
        }
    )
    fair_xgb.fit(X_train, y_train, protected_train)
    evaluate_detailed_metrics(fair_xgb, X_test, y_test, protected_test)

    print("\n=== Training Regular XGBoost Model ===")
    regular_xgb = xgb.XGBClassifier(
        max_depth=3,
        learning_rate=0.1,
        n_estimators=200,
        random_state=42,
        objective='binary:logistic'
    )
    regular_xgb.fit(X_train, y_train)
    evaluate_detailed_metrics(regular_xgb, X_test, y_test, protected_test)

if __name__ == "__main__":
    main()


=== Training Fair XGBoost Model ===
Overall Performance Metrics:
              precision    recall  f1-score   support

           0       0.53      1.00      0.69      1153
           1       0.00      0.00      0.00      1012

    accuracy                           0.53      2165
   macro avg       0.27      0.50      0.35      2165
weighted avg       0.28      0.53      0.37      2165


Fairness Metrics:
Disparate Impact: 1.662
Statistical Parity Difference: 0.203
Equal Opportunity Difference: 0.000
Average Odds Difference: 0.000

=== Training Regular XGBoost Model ===
Overall Performance Metrics:
              precision    recall  f1-score   support

           0       0.71      0.73      0.72      1153
           1       0.68      0.66      0.67      1012

    accuracy                           0.69      2165
   macro avg       0.69      0.69      0.69      2165
weighted avg       0.69      0.69      0.69      2165


Fairness Metrics:
Disparate Impact: 1.662
Statistical Parity Di

c:\Users\fsd_n\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [17:23:38] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0750514818a16474a-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\fsd_n\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fsd_n\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fsd_n\anaconda3\Lib\site-packages\sklearn\metri